In [2]:
%load_ext autoreload
%autoreload 2 

import pandas as pd
import numpy as np
import torch
import numpy as np
import sys

sys.path.insert(0, '..')
from decompose_lstm import DecomposedLSTM
from lstm_dictionary import Dictionary


In [3]:
model_path = "../../lstm/data/model/state_dict.pt"
vocab_path = "../../lstm/data/model/vocab.txt"

decomposed_model = DecomposedLSTM.from_pretrained(model_path, shapley_include_bias=False)
tokenizer = Dictionary(vocab_path)


In [4]:
number_df = pd.read_csv("nounpp.tsv", delimiter="\t")
max_length = 16

number_df["id"] = number_df["id"].apply(lambda x: int(x[2:]))

# only keep sentences that are plural/singular or singular/plural (distractor has different number)
number_df["subject_distractor_number"] = number_df["subject_distractor_number"].apply(
    lambda x: x if x == "singular_plural" or x == "plural_singular" else np.nan
)
number_df.dropna(inplace=True)

number_df["subject_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[0])
number_df["distractor_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[1])

number_df["verb"] = number_df["sentence"].apply(lambda x: " " + x.split(" ")[-1])
number_df["sentence"] = number_df["sentence"].apply(lambda x: " ".join(x.split(" ")[:-1]))

number_df = number_df.drop(
    columns=["subject_distractor_number"]
    ).pivot(index=["id", "subject_number", "distractor_number", "sentence"], columns=["correctness"], values=["verb"]).reset_index()

number_df


id subject_number distractor_number  \
correctness                                          
0             601       singular            plural   
1             602       singular            plural   
2             603       singular            plural   
3             604       singular            plural   
4             605       singular            plural   
...           ...            ...               ...   
1195         1796         plural          singular   
1196         1797         plural          singular   
1197         1798         plural          singular   
1198         1799         plural          singular   
1199         1800         plural          singular   

                                  sentence         verb                
correctness                                     correct         wrong  
0             The athlete behind the bikes     approves       approve  
1              The athlete behind the cars     inspires       inspire  
2              The athlete behind the cats      engages        engage  
3              The athlete behind the cats    remembers      remember  
4            The athlete behind the chairs     observes       observe  
...                                    ...          ...           ...  
1195               The women near the tree        greet        greets  
1196             The women near the window        avoid        avoids  
1197             The women near the window   discourage   discourages  
1198             The women near the window       engage       engages  
1199             The women near the window   understand   understands  

[1200 rows x 6 columns]

In [5]:
number_df["sentence_tokens"] = number_df["sentence"].apply(tokenizer.tokenize)
number_df["correct_token"] = number_df[("verb", "correct")].apply(lambda x: tokenizer.tokenize(x).item())
number_df["wrong_token"] = number_df[("verb", "wrong")].apply(lambda x: tokenizer.tokenize(x).item())

number_df.head()


id subject_number distractor_number  \
correctness                                         
0            601       singular            plural   
1            602       singular            plural   
2            603       singular            plural   
3            604       singular            plural   
4            605       singular            plural   

                                  sentence        verb             \
correctness                                    correct      wrong   
0             The athlete behind the bikes    approves    approve   
1              The athlete behind the cars    inspires    inspire   
2              The athlete behind the cats     engages     engage   
3              The athlete behind the cats   remembers   remember   
4            The athlete behind the chairs    observes    observe   

                                               sentence_tokens correct_token  \
correctness                                                                    
0            [tensor(146), tensor(21749), tensor(2230), ten...         42696   
1            [tensor(146), tensor(21749), tensor(2230), ten...         33476   
2            [tensor(146), tensor(21749), tensor(2230), ten...         18223   
3            [tensor(146), tensor(21749), tensor(2230), ten...          8671   
4            [tensor(146), tensor(21749), tensor(2230), ten...          8739   

            wrong_token  
correctness              
0                 11336  
1                 17370  
2                  3610  
3                  2030  
4                  8501

In [ ]:
sentence_len = 5
assert number_df["sentence_tokens"].apply(len).unique() == [sentence_len]


def split_pos_neg_contributions(logits):
    """
    shape: (num_contributions + 1 (bias), num_classes)
    """
   # put negative "positive" contributions in the negative category
    # put negative "negative" contributions in the positive category
    positive_mask = (logits > 0).astype(int)

    positive_logits = logits[..., 1] * positive_mask[..., 1]
    positive_logits -= logits[..., 0] * (1 - positive_mask[..., 0])

    negative_logits = logits[..., 0] * positive_mask[..., 0]
    negative_logits -= logits[..., 1] * (1 - positive_mask[..., 1])

    # assert (np.all(positive_logits >= 0))
    # assert (np.all(negative_logits >= 0))

    binary_logits = np.stack(
        [negative_logits, positive_logits], axis=-2
    )
    # print(binary_logits.shape)
    return binary_logits


def get_jumulet_contribution(sentence_tokens, correct_token, wrong_token):
    all_contributions = torch.empty((sentence_len + 1,2))

    for i in range(-1, sentence_len):
        beta_mask = torch.zeros(sentence_len,)
        init_in_beta = False
        if i == -1:
            # get contribution of the initial state
            init_in_beta = True
        else:
            # get contribution of the token at index i
            beta_mask[i] = 1

        with torch.no_grad():
            beta, gamma, bias = decomposed_model(
                sentence_tokens.unsqueeze(0),
                beta_mask.unsqueeze(0),
                init_in_beta=init_in_beta
                )
            # get last hidden states
            z = beta + gamma + bias
            contribution = beta / z
            c = contribution[0, -1, (correct_token, wrong_token)]
            
            all_contributions[i + 1] = c
    return all_contributions



### GCD w/ fixed bias

In [10]:
get_jumulet_contribution(
    torch.tensor([146, 21749,  2230,     3, 14159]),
    42696,
    11336
)


tensor([[[ 0.2439,  0.2071,  0.9643,  ..., -1.9202, -1.9483,  0.3526],
         [ 0.9491,  5.1321,  9.4497,  ..., -2.9997, -2.5698, -0.6521],
         [ 0.2316, -1.0468,  8.4410,  ..., -2.3390, -3.8668,  0.1466],
         [-2.9794,  1.0338,  3.7435,  ..., -2.9436, -4.0727, -0.6532],
         [ 1.0138,  6.9565, 11.0821,  ..., -2.7540, -3.0840, -1.4964]]])
tensor([[[ 0.2439,  0.2071,  0.9643,  ..., -1.9202, -1.9483,  0.3526],
         [ 0.9491,  5.1321,  9.4497,  ..., -2.9997, -2.5698, -0.6521],
         [ 0.2316, -1.0468,  8.4410,  ..., -2.3390, -3.8668,  0.1466],
         [-2.9794,  1.0338,  3.7435,  ..., -2.9436, -4.0727, -0.6532],
         [ 1.0138,  6.9565, 11.0821,  ..., -2.7540, -3.0840, -1.4964]]])
tensor([[[ 0.2439,  0.2071,  0.9643,  ..., -1.9202, -1.9483,  0.3526],
         [ 0.9491,  5.1321,  9.4497,  ..., -2.9997, -2.5698, -0.6521],
         [ 0.2316, -1.0468,  8.4410,  ..., -2.3390, -3.8668,  0.1466],
         [-2.9794,  1.0338,  3.7435,  ..., -2.9436, -4.0727, -0.6532],
  

tensor([[ 0.7557,  0.4462],
        [ 0.3445,  0.3268],
        [ 0.6733,  0.2016],
        [-0.4387, -0.3993],
        [ 0.0400, -0.0152],
        [ 0.2623,  0.8895]])

In [7]:
print(tokenizer.tokenize("The doctor near the dogs knows"))
print(tokenizer.tokenize("The doctor near the dogs know"))


tensor([ 146, 6022,  220,    3, 1041, 4814])
tensor([ 146, 6022,  220,    3, 1041, 2678])


In [8]:
get_jumulet_contribution(
    torch.tensor([ 146, 6022,  220,    3, 1041]),
    4814,
    2678
)


tensor([[ 0.2786,  0.1383],
        [ 0.0915,  0.0613],
        [ 0.3559,  0.2025],
        [-0.0963,  0.0339],
        [ 0.0789, -0.0953],
        [ 0.2852,  0.4062]])

In [9]:
number_df["jumulet_contribution_fixed"] = number_df.apply(
    lambda x: get_jumulet_contribution(
        x["sentence_tokens"][0],
        x["correct_token"][0],
        x["wrong_token"][0]
    ),
    axis=1
)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/4009221104.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["sentence_tokens"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/4009221104.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["correct_token"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/4009221104.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["w

In [10]:
# singular subject, plural distractor
sp_contributions = np.stack(
    number_df[number_df["distractor_number"] == "plural"]["jumulet_contribution_fixed"]
    )

# singular subject
ps_contributions = np.stack(
    number_df[number_df["distractor_number"] == "singular"]["jumulet_contribution_fixed"]
    )


In [11]:
sp_contributions.mean(0)


array([[ 0.43970898,  0.4404421 ],
       [ 0.23971453,  0.17095037],
       [ 0.20777053, -0.2277045 ],
       [-0.19155994,  0.07164335],
       [ 0.09507721, -0.00318193],
       [ 0.44742966,  0.79140973]], dtype=float32)

In [12]:
ps_contributions.mean(0)


array([[ 0.24841301,  1.0414895 ],
       [ 0.13884118,  0.43080363],
       [ 0.38182613, -0.8785911 ],
       [ 0.06271271, -0.35090494],
       [ 0.08335493,  0.49208227],
       [ 0.18639894,  0.98074573]], dtype=float32)

### GCD w/ shapely bias

In [ ]:
decomposed_model.shapley_include_bias = True
decomposed_model.generalized = True


In [14]:
with torch.no_grad():
    # activations for ". <eos>"
    init_phrase = torch.LongTensor([18, 19]).unsqueeze(0)
    embed = decomposed_model.model.encoder(init_phrase)
    _, hidden = decomposed_model.model.rnn(embed)


In [15]:
all_contributions = torch.empty((sentence_len + 1,2))
sentence_tokens = torch.tensor([ 146, 6022,  220,    3, 1041])
contributions = torch.zeros((6,6))
correct_token = 4814
wrong_token = 2678

for i in range(-1, sentence_len):
    beta_mask = torch.zeros(sentence_len,)
    init_in_beta = False
    if i == -1:
        # get contribution of the initial state
        init_in_beta = True
    else:
        # get contribution of the token at index i
        beta_mask[i] = 1

    with torch.no_grad():
        beta, gamma, bias = decomposed_model(
            sentence_tokens.unsqueeze(0),
            beta_mask.unsqueeze(0),
            init_in_beta=init_in_beta
            )
        z, _ = decomposed_model.model.forward(
            sentence_tokens.unsqueeze(0),
            hidden
        )
        # get last hidden states
        # z = beta + gamma + bias

        contribution = beta / z
        for j in range(max(i+1, 1), sentence_len):
            future_token = sentence_tokens[j].item()
            print(i, j, tokenizer.idx2word[future_token])
            print(contribution[0, j - 1, future_token])
            contributions[i + 1, j - 1] = contribution[0, j - 1, future_token]
        
        contributions[i+1, 4] = contribution[0, -1, correct_token]
        contributions[i+1, 5] = contribution[0, -1, wrong_token]




-1 1 doctor
tensor(0.2253, dtype=torch.float64)
-1 2 near
tensor(0.1425, dtype=torch.float64)
-1 3 the
tensor(-0.0316, dtype=torch.float64)
-1 4 dogs
tensor(0.5632, dtype=torch.float64)
0 1 doctor
tensor(0.4304, dtype=torch.float64)
0 2 near
tensor(0.1263, dtype=torch.float64)
0 3 the
tensor(-0.0108, dtype=torch.float64)
0 4 dogs
tensor(0.4810, dtype=torch.float64)
1 2 near
tensor(-0.0610, dtype=torch.float64)
1 3 the
tensor(0.0038, dtype=torch.float64)
1 4 dogs
tensor(0.6334, dtype=torch.float64)
2 3 the
tensor(0.1513, dtype=torch.float64)
2 4 dogs
tensor(0.0379, dtype=torch.float64)
3 4 dogs
tensor(0.0574, dtype=torch.float64)


In [16]:
contributions.numpy().round(2)


array([[ 0.23,  0.14, -0.03,  0.56,  0.49,  0.  ],
       [ 0.43,  0.13, -0.01,  0.48,  0.34,  0.07],
       [ 0.  , -0.06,  0.  ,  0.63,  0.53,  0.24],
       [ 0.  ,  0.  ,  0.15,  0.04,  0.1 ,  0.05],
       [ 0.  ,  0.  ,  0.  ,  0.06,  0.14, -0.29],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.33,  0.29]], dtype=float32)

In [17]:
get_jumulet_contribution(
    torch.tensor([ 146, 6022,  220,    3, 1041]),
    4814,
    2678
)


tensor([[ 0.4932,  0.0014],
        [ 0.3426,  0.0726],
        [ 0.5288,  0.2434],
        [ 0.0958,  0.0529],
        [ 0.1420, -0.2898],
        [ 0.3310,  0.2850]])

In [18]:
number_df["jumulet_contribution_all"] = number_df.apply(
    lambda x: get_jumulet_contribution(
        x["sentence_tokens"][0],
        x["correct_token"][0],
        x["wrong_token"][0]
    ),
    axis=1
)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2760458623.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["sentence_tokens"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2760458623.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["correct_token"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2760458623.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["w

In [19]:
# singular subject, plural distractor
sp_contributions = np.stack(
    number_df[number_df["distractor_number"] == "plural"]["jumulet_contribution_all"]
    )

# singular subject
ps_contributions = np.stack(
    number_df[number_df["distractor_number"] == "singular"]["jumulet_contribution_all"]
    )


In [20]:
sp_contributions.mean(0)


array([[ 0.7526021 ,  0.16763318],
       [ 0.50965893,  0.16434468],
       [ 0.48675588, -0.11615125],
       [ 0.03766717,  0.06272028],
       [ 0.24761301, -0.26764938],
       [ 0.55568933,  0.4488798 ]], dtype=float32)

In [21]:
ps_contributions.mean(0)


array([[ 0.12187713,  1.8369094 ],
       [ 0.04983244,  1.3322237 ],
       [ 0.5689856 , -0.1285119 ],
       [ 0.04693957,  0.23761767],
       [-0.08282404,  0.7737088 ],
       [-0.04266999,  1.1255189 ]], dtype=float32)

### CD w/ fixed bias


In [32]:
decomposed_model.shapley_include_bias = False
decomposed_model.generalized = False


In [34]:
get_jumulet_contribution(
    torch.tensor([ 146, 6022,  220,    3, 1041]),
    4814,
    2678
)


tensor([[ 6.7735e-02,  1.6436e-02],
        [ 2.6688e-02,  2.5698e-02],
        [ 6.7182e-02,  1.2396e-01],
        [-6.5648e-05,  1.6064e-01],
        [ 1.2273e-01,  1.0233e-01],
        [ 1.7873e-01,  4.8859e-01]])

In [35]:
number_df["murdoch_contribution_all"] = number_df.apply(
    lambda x: get_jumulet_contribution(
        x["sentence_tokens"][0],
        x["correct_token"][0],
        x["wrong_token"][0]
    ),
    axis=1
)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["sentence_tokens"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["correct_token"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["w

In [36]:
# singular subject, plural distractor
sp_contributions = np.stack(
    number_df[number_df["distractor_number"] == "plural"]["murdoch_contribution_all"]
    )

# singular subject
ps_contributions = np.stack(
    number_df[number_df["distractor_number"] == "singular"]["murdoch_contribution_all"]
    )


In [37]:
sp_contributions.mean(0)


array([[ 0.11075391,  0.52858996],
       [ 0.05287185,  0.0493324 ],
       [ 0.04832102, -0.02005999],
       [-0.13502298, -0.10924449],
       [ 0.1682476 ,  0.07372447],
       [ 0.23354782,  1.0271769 ]], dtype=float32)

In [38]:
ps_contributions.mean(0)


array([[ 0.15032065,  0.19854659],
       [ 0.00806151,  0.09250602],
       [ 0.08498269, -0.15722825],
       [-0.05861461, -0.19921213],
       [-0.03069262,  0.41683728],
       [ 0.01059884,  0.76912606]], dtype=float32)

### CD w/ Shapley bias

In [39]:
decomposed_model.shapley_include_bias = True
decomposed_model.generalized = False


In [40]:
number_df["murdoch_contribution_all"] = number_df.apply(
    lambda x: get_jumulet_contribution(
        x["sentence_tokens"][0],
        x["correct_token"][0],
        x["wrong_token"][0]
    ),
    axis=1
)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["sentence_tokens"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["correct_token"][0],
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_87779/2305937806.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["w

In [41]:
# singular subject, plural distractor
sp_contributions = np.stack(
    number_df[number_df["distractor_number"] == "plural"]["murdoch_contribution_all"]
    )

# singular subject
ps_contributions = np.stack(
    number_df[number_df["distractor_number"] == "singular"]["murdoch_contribution_all"]
    )


In [42]:
sp_contributions.mean(0)


array([[0.2558565 , 0.08301068],
       [0.18312702, 0.20157708],
       [0.19482486, 0.09758325],
       [0.18275078, 0.02052693],
       [0.19952725, 0.18607761],
       [0.10038027, 0.3066745 ]], dtype=float32)

In [43]:
ps_contributions.mean(0)


array([[ 0.06104723, -7.9162555 ],
       [ 0.06252284,  0.37209895],
       [-0.61590654,  0.8521732 ],
       [ 0.0444878 ,  0.31143522],
       [ 0.09013457,  0.62945056],
       [ 0.04749724,  0.31624857]], dtype=float32)